In [17]:
from convexness.convexness import *
import glob
import os
import open3d as o3d
import pandas as pd
from tqdm import tqdm

In [18]:
n_random_samples = 800
min_fragments = 3
max_fragments = 8

In [19]:
def process_our_folder(folder):
    target_extension = '.stl'
    metadata_pattern = '*_mesh.txt'
    path = folder + '*/' + metadata_pattern
    mesh_files = glob.glob(folder + '*/' + metadata_pattern, recursive=True)
    
    #
    global_df = None
    
    for metadata_file in mesh_files:
        df = pd.read_csv(metadata_file, sep='\t', header=0, index_col=False)
        
        # maintain only a subset of the columns
        df = df[['Filename', 'Percentage', 'Vertices', 'Faces']]
        if global_df is None:
            global_df = df
        else:
            global_df = pd.concat([global_df, df], ignore_index=True)
            
    # filter files by target extension
    global_df = global_df[global_df['Filename'].str.contains(target_extension)]
            
    # create a new column with the number of fragments
    global_df['Fragments'] = global_df['Filename'].apply(lambda x: int(x.split('f_')[0].split('_')[-1]))
    # select n random samples
    global_df = global_df[max_fragments >= global_df['Fragments']]  
    global_df = global_df[min_fragments <= global_df['Fragments']]
    
    print('Number of fragments:', global_df.shape[0])
    
    global_df = global_df.sample(n=n_random_samples)

    # iterate to calculate the convexness
    for index, row in tqdm(global_df.iterrows(), total=global_df.shape[0]):
        filename = row['Filename']
        filename = filename.replace('E:/Fragments/', 'D:/allopezr/Fragments/vessels_200_obj_ply_no_zipped/')
        
        mesh = trimesh.load_mesh(filename)
        # normalize the mesh
        mesh.vertices -= mesh.vertices.mean(axis=0)
        mesh.vertices /= np.abs(mesh.vertices).max()
        
        try:
            convexness = measure_convexness(mesh, 500, 1)
            global_df.loc[index, 'Convexness'] = convexness
        except:
            print('Error:', filename)
            global_df = global_df.drop(index)
        
    return global_df

In [20]:
def process_voronoi_folder(folder):
    target_extension = '.obj'
    files = glob.glob(folder + '*/*' + target_extension, recursive=True)
    # get number of fragments as the first number in the filename before '_'
    global_df = pd.DataFrame(columns=['Filename', 'Fragments'])
    for file in files:
        file_name = os.path.basename(file)
        fragments = int(file_name.split('_')[0])
        global_df = pd.concat([global_df, pd.DataFrame({'Filename': [file], 'Fragments': [fragments]})], ignore_index=True)
        
    # filter by the number of fragments
    global_df = global_df[max_fragments >= global_df['Fragments']]
    global_df = global_df[min_fragments <= global_df['Fragments']] 
    
    # random sampling
    print('Number of fragments:', global_df.shape[0])
    random_rows = global_df.sample(n=n_random_samples)
    for index, row in tqdm(random_rows.iterrows()):
        filename = row['Filename']
        mesh = trimesh.load_mesh(filename)
        
        # normalize the mesh
        mesh.vertices -= mesh.vertices.mean(axis=0)
        mesh.vertices /= np.abs(mesh.vertices).max()
        
        try:
            # quadric decimation
            #mesh = mesh.simplify_quadric_decimation(10000)
            convexness = measure_convexness(mesh, 500, 1)
            global_df.loc[index, 'Convexness'] = convexness
        except:
            print('Error:', filename)
            global_df = global_df.drop(index)
        
    # remove rows with NaN values
    global_df = global_df.dropna()
    
    return global_df

In [21]:
def process_sellan_folder(folder):
    # find subfolders
    subfolders = glob.glob(folder + '*/*/')
    global_df = pd.DataFrame(columns=['Filename', 'Fragments'])
    
    for subfolder in subfolders:
        # find obj inside the subfolder
        files = glob.glob(subfolder + '*.obj')
        n_fragments = len(files)
        
        if min_fragments<= n_fragments <= max_fragments:          
            for file in files:
                global_df = pd.concat([global_df, pd.DataFrame({'Filename': [file], 'Fragments': [n_fragments]})], ignore_index=True)
            
    print('Number of fragments:', global_df.shape[0])
            
    # read the mesh
    random_rows = global_df.sample(n=n_random_samples)
    for index, row in tqdm(random_rows.iterrows()):
        filename = row['Filename']
        mesh = trimesh.load_mesh(filename)
        # normalize the mesh
        mesh.vertices -= mesh.vertices.mean(axis=0)
        mesh.vertices /= np.abs(mesh.vertices).max()
        
        try:
            convexness = measure_convexness(mesh, 500, 1)
            global_df.loc[index, 'Convexness'] = convexness
        except:
            print('Error:', filename)
            global_df = global_df.drop(index)
        
    # remove rows with NaN values
    global_df = global_df.dropna()
    
    return global_df

In [ ]:
#folders = ['D:/allopezr/Fragments/Vessels_200_Voronoi/', 'D:/allopezr/Fragments/vessels_200_obj_ply_no_zipped/vessels_200_obj_ply/']

folders = ['D:/allopezr/Fragments/Artifacts_200_CellFracture/', 'D:/allopezr/Fragments/Artifacts_200_ours_stl/', 'D:/allopezr/Datasets/BreakingBad/artifact/']

#folders = ['D:/allopezr/Datasets/BreakingBad/artifact_compressed/']
results = []

for folder in tqdm(folders):
    print(folder)
    
    if 'CellFracture' in folder:
        global_df = process_voronoi_folder(folder)
    elif 'BreakingBad' in folder:
        global_df = process_sellan_folder(folder)
    else:
        global_df = process_our_folder(folder) 
        
    results.append(global_df[['Fragments', 'Convexness']])

  0%|          | 0/3 [00:00<?, ?it/s]

D:/allopezr/Fragments/Artifacts_200_CellFracture/
Number of fragments: 812



0it [00:00, ?it/s]
1it [00:00,  1.16it/s]
2it [00:01,  1.30it/s]
3it [00:02,  1.40it/s]
4it [00:02,  1.43it/s]
5it [00:04,  1.02s/it]
6it [00:05,  1.16it/s]
7it [00:05,  1.27it/s]
8it [00:06,  1.39it/s]
9it [00:06,  1.41it/s]
10it [00:07,  1.41it/s]
11it [00:08,  1.46it/s]
12it [00:08,  1.47it/s]
13it [00:09,  1.52it/s]
14it [00:10,  1.54it/s]
15it [00:10,  1.57it/s]
16it [00:11,  1.61it/s]
17it [00:12,  1.58it/s]
18it [00:12,  1.51it/s]
19it [00:13,  1.55it/s]
20it [00:14,  1.53it/s]
21it [00:14,  1.60it/s]
22it [00:14,  1.98it/s]
23it [00:15,  1.78it/s]
24it [00:16,  1.84it/s]
25it [00:16,  1.74it/s]
26it [00:17,  1.57it/s]
27it [00:17,  1.64it/s]
28it [00:18,  1.62it/s]
29it [00:19,  1.51it/s]
30it [00:20,  1.51it/s]
31it [00:20,  1.39it/s]
32it [00:21,  1.42it/s]
33it [00:22,  1.45it/s]
34it [00:22,  1.59it/s]
35it [00:23,  1.51it/s]
36it [00:24,  1.43it/s]
37it [00:24,  1.43it/s]
38it [00:25,  1.56it/s]
39it [00:26,  1.59it/s]
40it [00:26,  1.56it/s]
41it [00:27,  1.64it/s]
42it 

Error: D:/allopezr/Fragments/Artifacts_200_CellFracture\118\5_118.compressed_mesh_cell.011.obj



77it [00:47,  2.14it/s]
78it [00:47,  1.95it/s]
79it [00:48,  2.03it/s]
80it [00:48,  2.07it/s]
81it [00:49,  1.87it/s]D:\allopezr\MeshFragments\Python\Visualizer\convexness\convexness.py:57: RuntimeWarning: invalid value encountered in divide
  T = (locations[:, 0] - ray_origins[:, 0]) / ray_directions[:, 0]

82it [00:49,  1.98it/s]
83it [00:50,  1.95it/s]
84it [00:50,  1.97it/s]
85it [00:51,  1.83it/s]
86it [00:51,  1.90it/s]
87it [00:52,  1.75it/s]
88it [00:53,  1.64it/s]
89it [00:53,  1.70it/s]
90it [00:54,  1.56it/s]
91it [00:55,  1.59it/s]
92it [00:55,  1.62it/s]
93it [00:56,  1.59it/s]
94it [00:56,  1.68it/s]
95it [00:57,  1.64it/s]
96it [00:58,  1.72it/s]
97it [00:58,  1.70it/s]
98it [00:59,  1.71it/s]
99it [00:59,  1.80it/s]

...


262it [02:33,  2.11it/s]
263it [02:34,  1.96it/s]
264it [02:35,  1.86it/s]
265it [02:35,  1.77it/s]
266it [02:36,  1.85it/s]
267it [02:36,  2.07it/s]
268it [02:36,  2.05it/s]
269it [02:37,  1.77it/s]
270it [02:38,  1.71it/s]
271it [02:39,  1.58it/s]
272it [02:39,  1.76it/s]
273it [02:40,  1.73it/s]
274it [02:40,  1.77it/s]
275it [02:41,  1.76it/s]
276it [02:42,  1.54it/s]
277it [02:42,  1.79it/s]
278it [02:42,  1.86it/s]
279it [02:43,  1.71it/s]
280it [02:44,  1.82it/s]
281it [02:44,  1.65it/s]
282it [02:45,  1.69it/s]
283it [02:45,  1.73it/s]
284it [02:46,  1.67it/s]
285it [02:47,  1.72it/s]
286it [02:47,  1.71it/s]
287it [02:48,  1.75it/s]
288it [02:48,  1.78it/s]
289it [02:49,  1.70it/s]
290it [02:50,  1.72it/s]
291it [02:50,  1.54it/s]
292it [02:51,  1.61it/s]
293it [02:51,  1.93it/s]
294it [02:52,  2.05it/s]
295it [02:52,  2.06it/s]
296it [02:53,  1.86it/s]
297it [02:53,  1.96it/s]D:\allopezr\MeshFragments\Python\Visualizer\convexness\convexness.py:57: RuntimeWarning: invalid val

Error: D:/allopezr/Fragments/Artifacts_200_CellFracture\118\5_118.compressed_mesh_cell.008.obj



405it [03:58,  2.20it/s]
406it [03:59,  1.77it/s]
407it [04:00,  1.67it/s]
408it [04:00,  1.67it/s]
409it [04:01,  1.63it/s]
410it [04:02,  1.52it/s]
411it [04:02,  1.60it/s]
412it [04:03,  1.65it/s]
413it [04:04,  1.59it/s]
414it [04:04,  1.52it/s]
415it [04:05,  1.48it/s]
416it [04:06,  1.57it/s]
417it [04:06,  1.50it/s]
418it [04:07,  1.56it/s]
419it [04:08,  1.62it/s]
420it [04:08,  1.52it/s]
421it [04:09,  1.64it/s]
422it [04:10,  1.51it/s]
423it [04:10,  1.57it/s]
424it [04:11,  1.57it/s]
425it [04:11,  1.57it/s]
426it [04:12,  1.56it/s]
427it [04:13,  1.63it/s]
428it [04:13,  1.71it/s]
429it [04:14,  1.75it/s]
430it [04:14,  1.69it/s]
431it [04:15,  1.49it/s]
432it [04:16,  1.47it/s]
433it [04:17,  1.46it/s]
434it [04:17,  1.41it/s]
435it [04:18,  1.59it/s]
436it [04:18,  1.65it/s]
437it [04:19,  1.54it/s]
438it [04:20,  1.52it/s]
439it [04:20,  1.98it/s]
440it [04:20,  2.39it/s]
441it [04:21,  2.16it/s]
442it [04:21,  1.90it/s]
443it [04:22,  1.88it/s]
444it [04:22,  1.91it/s]

Error: D:/allopezr/Fragments/Artifacts_200_CellFracture\88\7_88.compressed_mesh_cell.008.obj



460it [04:31,  2.16it/s]
461it [04:32,  1.98it/s]
462it [04:33,  1.81it/s]
463it [04:33,  1.65it/s]
464it [04:34,  1.50it/s]
465it [04:35,  1.62it/s]
466it [04:35,  1.58it/s]
467it [04:36,  1.61it/s]
468it [04:37,  1.66it/s]

In [ ]:
results_copy = results.copy()

In [ ]:
# calculate only for sellan
global_df = process_sellan_folder('D:/allopezr/Datasets/BreakingBad/artifact/')

In [ ]:
# substitute this result with the previous one
results[2] = global_df[['Fragments', 'Convexness']]

In [ ]:
import matplotlib.pyplot as plt

# Change matplotlib style
plt.style.use('default')

font_mapping = {'family': 'Adobe Devanagari', 'weight': 'normal', 'size': 19}
plt.rc('font', **font_mapping)

In [ ]:
# render the results
import matplotlib.pyplot as plt

legends = ['Voronoi', 'Our work', 'Sellán et al.']

plt.figure(figsize=(8, 5))

# average convexness per number of fragments
for idx, (df, legend) in enumerate(zip(results, legends)):  
    df = df.dropna()
    mean = df.groupby('Fragments').mean()
    std = df.groupby('Fragments').std()  
    
    # box plot
    plt.errorbar(mean.index, mean['Convexness'], yerr=std['Convexness'], fmt='o-', label=legend, capsize=10)
    
plt.legend(frameon=False, loc='upper center', bbox_to_anchor=(0.5, 1.2), ncol=3)
plt.xlabel('Fragments')
plt.ylabel('Convexity')
plt.xlim(3 - 0.14, max_fragments + 0.14)
plt.ylim(.0, 1.0)
plt.tight_layout()
plt.savefig('convexness.png', dpi=300)
plt.show()

In [ ]:
# percentil 75, 50, 25
for df in results:
    print(df['Convexness'].quantile([0.25, 0.5, 0.75]))